# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [4]:
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      5735 non-null   int64  
 1   ALQ101    5208 non-null   float64
 2   ALQ110    1731 non-null   float64
 3   ALQ130    3379 non-null   float64
 4   SMQ020    5735 non-null   int64  
 5   RIAGENDR  5735 non-null   int64  
 6   RIDAGEYR  5735 non-null   int64  
 7   RIDRETH1  5735 non-null   int64  
 8   DMDCITZN  5734 non-null   float64
 9   DMDEDUC2  5474 non-null   float64
 10  DMDMARTL  5474 non-null   float64
 11  DMDHHSIZ  5735 non-null   int64  
 12  WTINT2YR  5735 non-null   float64
 13  SDMVPSU   5735 non-null   int64  
 14  SDMVSTRA  5735 non-null   int64  
 15  INDFMPIR  5134 non-null   float64
 16  BPXSY1    5401 non-null   float64
 17  BPXDI1    5401 non-null   float64
 18  BPXSY2    5535 non-null   float64
 19  BPXDI2    5535 non-null   float64
 20  BMXWT     5666 non-null   floa

In [9]:
sample = (da['RIDAGEYR']).between(35,50, inclusive = True) & (da['RIAGENDR'] == 2)
women = da[sample]
women

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
34,83799,NaN,NaN,NaN,2,2,37,2,1.0,4.0,...,110.0,72.0,66.6,161.6,25.5,NaN,NaN,NaN,NaN,2.0
50,83828,1.0,NaN,2.0,2,2,39,1,2.0,3.0,...,100.0,62.0,71.3,162.0,27.2,36.8,34.6,29.1,94.6,NaN
52,83832,2.0,1.0,4.0,2,2,50,1,2.0,1.0,...,NaN,NaN,105.9,157.7,42.6,29.2,35.0,40.7,129.1,NaN
55,83837,2.0,2.0,NaN,2,2,45,1,1.0,2.0,...,114.0,68.0,77.5,148.3,35.2,30.5,34.0,34.4,107.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5658,93568,1.0,NaN,1.0,1,2,46,3,1.0,2.0,...,128.0,84.0,85.5,152.4,36.8,26.0,33.5,34.7,116.5,2.0
5685,93612,2.0,2.0,NaN,2,2,36,5,1.0,5.0,...,112.0,74.0,65.7,162.4,24.9,37.2,36.0,29.8,89.0,2.0
5689,93619,NaN,NaN,NaN,2,2,44,5,2.0,2.0,...,118.0,78.0,51.9,149.3,23.3,31.9,32.0,26.2,81.4,1.0
5721,93676,1.0,NaN,2.0,2,2,35,4,1.0,5.0,...,114.0,76.0,92.2,161.7,35.3,41.5,37.5,38.9,110.9,2.0


In [17]:
#married status
married_women = women[(women['DMDMARTL'] == 1)]
married_women


,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
34,83799,NaN,NaN,NaN,2,2,37,2,1.0,4.0,...,110.0,72.0,66.6,161.6,25.5,NaN,NaN,NaN,NaN,2.0
50,83828,1.0,NaN,2.0,2,2,39,1,2.0,3.0,...,100.0,62.0,71.3,162.0,27.2,36.8,34.6,29.1,94.6,NaN
55,83837,2.0,2.0,NaN,2,2,45,1,1.0,2.0,...,114.0,68.0,77.5,148.3,35.2,30.5,34.0,34.4,107.6,2.0
61,83851,2.0,1.0,1.0,1,2,37,3,1.0,3.0,...,122.0,74.0,85.1,155.3,35.3,32.5,33.6,36.1,106.5,1.0
62,83853,NaN,NaN,NaN,2,2,49,3,1.0,3.0,...,116.0,84.0,76.1,166.7,27.4,39.2,38.6,32.6,88.7,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,93488,2.0,1.0,1.0,2,2,47,4,1.0,4.0,...,104.0,66.0,57.9,158.0,23.2,37.2,36.0,28.1,79.4,2.0
5627,93516,NaN,NaN,NaN,2,2,43,2,1.0,4.0,...,124.0,78.0,82.7,165.3,30.3,38.8,38.8,32.2,102.2,NaN
5685,93612,2.0,2.0,NaN,2,2,36,5,1.0,5.0,...,112.0,74.0,65.7,162.4,24.9,37.2,36.0,29.8,89.0,2.0
5689,93619,NaN,NaN,NaN,2,2,44,5,2.0,2.0,...,118.0,78.0,51.9,149.3,23.3,31.9,32.0,26.2,81.4,1.0


In [18]:

notmarried_women = women[~(women['DMDMARTL'] == 1)]
notmarried_women


,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
52,83832,2.0,1.0,4.0,2,2,50,1,2.0,1.0,...,NaN,NaN,105.9,157.7,42.6,29.2,35.0,40.7,129.1,NaN
58,83845,1.0,NaN,NaN,1,2,44,4,1.0,1.0,...,116.0,78.0,133.3,171.5,45.3,37.3,35.7,48.7,NaN,2.0
100,83911,1.0,NaN,1.0,2,2,43,4,1.0,4.0,...,126.0,76.0,91.1,172.3,30.7,40.4,38.2,34.6,101.6,NaN
127,83958,2.0,1.0,2.0,1,2,47,4,1.0,3.0,...,148.0,76.0,58.6,160.6,22.7,36.8,35.2,28.0,77.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,93386,1.0,NaN,2.0,2,2,50,4,1.0,4.0,...,134.0,84.0,91.3,160.5,35.4,43.0,38.5,40.0,112.9,2.0
5582,93434,1.0,NaN,1.0,1,2,36,4,1.0,3.0,...,126.0,70.0,58.3,162.5,22.1,37.6,35.8,26.9,75.5,2.0
5623,93511,2.0,2.0,NaN,2,2,40,4,1.0,3.0,...,148.0,108.0,98.8,167.3,35.3,37.6,38.5,44.6,129.7,1.0
5658,93568,1.0,NaN,1.0,1,2,46,3,1.0,2.0,...,128.0,84.0,85.5,152.4,36.8,26.0,33.5,34.7,116.5,2.0


In [22]:
married_college = married_women[(married_women['DMDEDUC2'] == 5)]
married_college
percentage_married_college = len(married_college)/len(married_women)
percentage_married_college

0.36080178173719374

In [23]:
notmarried_college = notmarried_women[(notmarried_women['DMDEDUC2'] == 5)]
notmarried_college
percentage_notmarried_college = len(notmarried_college)/len(notmarried_women)
percentage_notmarried_college

0.21301775147928995

__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

## Question 2

Construct 95% confidence intervals for the proportion of smokers who are female, and for the proportion of smokers who are male.  Then construct a 95% confidence interval for the difference between these proportions.

In [ ]:
# enter your code here

__Q2a.__ Discuss why it may be relevant to report the proportions of smokers who are female and male, and contrast this to reporting the proportions of males and females who smoke.

__Q2b.__ How does the width of the confidence interval for the difference of the two proportions compare to the widths of the confidence intervals for each proportion separately?

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [ ]:
# enter your code here

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [ ]:
# enter your code here

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [ ]:
# enter code here

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?


__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [ ]:
# insert your code here

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

In [1]:
# insert your code here

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?